In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display, HTML

from sklearn.datasets import fetch_20newsgroups 

newsgroup = fetch_20newsgroups()

# les messages
messages = newsgroup['data']

# répartition des messages
solution = pd.Series(newsgroup['target'])
group_df = pd.DataFrame(solution.value_counts().sort_index(), columns=['msg_count'])
group_df.insert(0, 'group_names', newsgroup['target_names'])

print(group_df)
print(len(messages))

                 group_names  msg_count
0                alt.atheism        480
1              comp.graphics        584
2    comp.os.ms-windows.misc        591
3   comp.sys.ibm.pc.hardware        590
4      comp.sys.mac.hardware        578
5             comp.windows.x        593
6               misc.forsale        585
7                  rec.autos        594
8            rec.motorcycles        598
9         rec.sport.baseball        597
10          rec.sport.hockey        600
11                 sci.crypt        595
12           sci.electronics        591
13                   sci.med        594
14                 sci.space        593
15    soc.religion.christian        599
16        talk.politics.guns        546
17     talk.politics.mideast        564
18        talk.politics.misc        465
19        talk.religion.misc        377
11314


In [17]:
# fonction qui va séparer les données de chaque message
def format_message(str) -> dict['headers':dict, 'message':str]: 

    lines = str.split('\n')

# 1. GET HEADERS
    headers = {}
    message_index = 0
    last_header = ''
    for i,l in enumerate(lines) :
        # Détetcter le premier saut de ligne qui represente le début du message
        if l == '' :
            message_index = i
            break
        
        head = l.split(':',1)
        # gestion des exceptions
        if len(head) < 2 :
            # aucune entête
            if last_header == '' :
                if 'TRASH' not in headers :  headers['TRASH'] = ''
                headers['TRASH'] += (head[0])
            # entête multi-ligne
            else : 
                headers[last_header] += (head[0])
        # cas normal
        else :
            last_header = head[0]
            headers[head[0]] = head[1][1:]

# 2. GET MESSAGE
    message = '\n'.join(lines[message_index:])

    return {'headers': headers, 'message': message}

In [18]:
for k,v in format_message(messages[690])['headers'].items():
    print(k,':',v)

TRASH :  cs.utexas.edu!uunet!olivea!sgigate!sgiblab!adagio.panasonic.com!nntp-server.caltech.edu!keith
Subject : Re: <Political Atheists?
From : keith@cco.caltech.edu (Keith Allan Schneider) <930401.112329.0u1.rusnews.w165w@mantis.co.uk> <11710@vice.ICO.TEK.COM>
Organization : California Institute of Technology, Pasadena
NNTP-Posting-Host : lloyd.caltech.edu
Lines : 17


In [19]:
from pprint import pprint

# Faire la liste de tous les headers pour voir haha
headers_list = {}

for i, msg in enumerate(messages):
    headers = format_message(msg)['headers']
    for header in headers.keys():
        if header not in headers_list:
            headers_list[header] = 1
        else:
            headers_list[header] += 1

print(len(headers_list))

# filtrage des entêtes les plus présentes
limite = 999
main_head = dict(filter(lambda a: True if a[1] > limite else False, headers_list.items()))

pprint(main_head)


119
{'Distribution': 2533,
 'From': 11314,
 'Lines': 11276,
 'NNTP-Posting-Host': 2311,
 'Nntp-Posting-Host': 2453,
 'Organization': 10841,
 'Reply-To': 1720,
 'Subject': 11314}


In [20]:
# créer un dataframe qui va contenir ces headers pour aider à catégoriser les messages
# laisser le texte et la signature en brut

# liste des messages
mess_list = []
# dict des headers, chaque valeur est une liste de la longueur du tableau
head_dict = {}
for h in main_head.keys():
    head_dict[h] = []

# remplissage de ces objets
for msg in messages:
    mf = format_message(msg)
    # garniture des messages
    mess_list.append(mf['message'][1:])
    # garniture des headers
    to_fill = list(head_dict.keys()) # réduis à mesure que les valeurs sont renseignées
    for header, value in mf['headers'].items():
        if header in head_dict:
            head_dict[header].append(value)
            to_fill.remove(header)
    for h in to_fill: head_dict[h].append(None) # remplis les valeurs non renseignées

df = pd.DataFrame.from_dict(head_dict)
df['Message'] = mess_list

# export vers un format de stockage (le csv pose un problème, je ne sais pas quel séparateur utiliser)
df.to_json('./newsgroup.json')

Je crois que je comprends ce qu’il faut faire : 
(c’est faux)
1. faire une classification des textes du subset train avec un algo genre kmeans (pas retenu les autres)\
Donc on a pas besoin de faire un trainTestSplit en fait ?\
Ou alors on travaille avec toutes les données et on split nous même, pourquoi pas.\
Mais la grande question c’est si on peut fournir le Y ?
On fait de l’apprentissage supervisé ou non ?
2. vectoriser chaque groupe obtenu. Idéalement avec les deux methodes : CountVectorizer et TfIDFVectorizer\
Question : Comment les attribuer à un groupe ? 
3. diviser les textes du groupe de test selon leurs catégories (avec subset='test')
4. les vectoriser aussi ?
5. comparer les deux (je vois pas comment)

In [21]:
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.cluster import KMeans

# copier le code de test de kmeans

def bench_k_means(estimator, data, y=None):
    estimator.fit(data)
    scores = [
        "adjusted_rand_score",
        "adjusted_mutual_info_score",
        "silhouette_score",
        "homogeneity_score",
        "completeness_score",
        "v_measure_score",
    ]

    results = []
    for s in scores:
        if s not in ["silhouette_score"]:
            # with ground-truth
            # Yeah !
            score = getattr(metrics, s)(y, estimator.labels_)
        elif y is not None:
            # with NO ground-truth
            score = getattr(metrics, s)(data, estimator.labels_)
        else:
            continue
            
        results.append(score)
        
    return pd.DataFrame([results], columns=scores, index=["result"])

In [97]:
# les données
Xtrain, Ytrain = fetch_20newsgroups(return_X_y=True, remove=('headers', 'footers', 'quotes'))
groups = fetch_20newsgroups()['target_names']

# Bien sur les phrases doivent être vectorisées
c_vect = CountVectorizer(max_features=20, stop_words='english')
X_vec = c_vect.fit_transform(Xtrain[:20])
feat_n = c_vect.get_feature_names_out()
print(feat_n)
print(X_vec.toarray())

['20' '25' 'board' 'capability' 'car' 'does' 'don' 'flights' 'info'
 'insurance' 'new' 'option' 'parent' 'power' 'scsi' 'ssf' 'tiff' 've'
 'year' 'years']
[[ 0  0  0  0  4  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  2  0  2  0  1  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  2  1  0  2  0  0  0  0  0 25  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  1  0  0  0  0  0  0  0  0  0  0  0  3  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  1  0  0  0  0  0  8  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  1  9  0  1  0  9 

In [112]:
vectorizer = CountVectorizer(
    max_features=5000, 
    min_df=0.01, 
    max_df=0.8, 
    stop_words='english'
    )
X_vector = vectorizer.fit_transform(Xtrain)

model = KMeans(n_clusters=20, n_init='auto', max_iter=1000)
# x_categorized = model.fit_transform(X_vector)
model.fit(X_vector)
x_trans = model.transform(X_vector)
x_categorized = model.predict(X_vector)

print(len(x_categorized))
print(x_categorized[666])
print(x_categorized[888])

print(x_trans[666])
print(x_trans[888])

11314
0
0
[  5.18963711 126.3159863   33.30676495 342.98250684 164.35192593
 382.10731477 125.24376232 167.02162202 136.74063039 165.57777629
 201.22375605 117.15614581 158.21504353 224.17069389 228.08331811
 130.01153795 251.64657756 113.75265565 240.42462436 207.46083968]
[ 10.51672466 125.8308192   31.76164558 342.52299193 161.43282056
 382.2420699  124.7196857  165.99364111 134.98148021 164.43539765
 201.45967338 117.55025521 157.87019985 224.18184583 226.98237817
 130.56798995 252.91302853 114.95796333 240.59301736 206.71477935]


In [113]:
main_words_by_category = {}

for i,cluster in enumerate(model.cluster_centers_):
    main_words = cluster.argsort()[-10:]
    main_words_by_category[f'Cluster{i}'] = []
    for word in main_words:
        main_words_by_category[f'Cluster{i}'].append(vectorizer.get_feature_names_out()[word])
    
pprint(main_words_by_category)

{'Cluster0': ['good',
              'use',
              'does',
              'time',
              'think',
              'know',
              'don',
              'just',
              'people',
              'like'],
 'Cluster1': ['ftp',
              'version',
              'file',
              'com',
              'image',
              'data',
              'use',
              'window',
              'available',
              'edu'],
 'Cluster10': ['28', '24', 'team', '29', '31', '33', '32', '34', '35', '25'],
 'Cluster11': ['80',
               '15',
               '10',
               '20',
               '25',
               '50',
               'appears',
               '40',
               'dos',
               '00'],
 'Cluster12': ['said',
               'groups',
               'don',
               'going',
               'll',
               'options',
               'package',
               'think',
               'president',
               'ms'],
 'Cluster13': 

OK.
Le vectorizer fait une liste des mots importants
comment il fait pour décider de l’importance ça je ne sais pas
Probablement que c’est simplement la fréquence des mots, mais cela doit être configurable
Puis on peut avoir le compte de chacun de ces mots.

Essayons de faire le pseudo code grossier des étapes : 

Séparation des données
vectorisation de chaque categorie

In [47]:
categories = fetch_20newsgroups().target_names
posts = {}
for cat in categories:
    posts[cat] = fetch_20newsgroups(remove=('headers', 'footers', 'quotes')).data


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']